In [1]:
import os
import cv2
import math
from ultralytics import YOLO
from flask import Flask, render_template, Response, request, jsonify

In [2]:
def make_pred(classNames, img, results):
    for result in results:
        boxes = result.boxes
        for box in boxes:
            conf = math.ceil((box.conf[0] * 100)) / 100
            if conf > 0.5:
                x1, y1, x2, y2 = box.xyxy[0].int().tolist()
                cv2.rectangle(img, (x1, y1), (x2, y2), (0, 255, 0), 3)
                cls = int(box.cls[0])
                class_name = classNames[cls]
                label = f'{class_name} : {conf}'
                t_size = cv2.getTextSize(label, 0, fontScale=1, thickness=2)[0]
                c2 = x1 + t_size[0], y1 - t_size[1] - 3
                cv2.rectangle(img, (x1,y1), c2, [0,255,0], -1, cv2.LINE_AA)
                cv2.putText(img, label, (x1, y1-2), 0, 1, [255,255,255], thickness=1, lineType=cv2.LINE_AA)

In [3]:
def image_detect(image_path):
    img   = cv2.imread(image_path)
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue","blacktip","whitetip"]
    
    if img is None:
        return None
    
    results = model(img)
    make_pred(classNames, img, results)
                
    result_path = image_path.replace('uploads', 'results')
    cv2.imwrite(result_path, img)
    
    return result_path

In [4]:
def video_detection(path):
    cap   = cv2.VideoCapture(path)
    model = YOLO("runs/detect/train/weights/best.pt")
    classNames = ["bouledogue", "blacktip", "whitetip"]
    
    while True:
        success, img = cap.read()
        results = model(img, stream=True)
        make_pred(classNames, img, results)        
        yield img

In [5]:
def generate_frames(path = ''):
    yolo_output = video_detection(path)
    for detection in yolo_output :
        ref, buffer = cv2.imencode('.jpg', detection)
        frame = buffer.tobytes()
        yield (b'--frame\r\n' b'Content-Type: image/jpeg\r\n\r\n' + frame + b'\r\n')

In [6]:
app = Flask(__name__) 

app.config['UPLOAD_FOLDER'] = 'static/uploads/'
app.config['RESULT_FOLDER'] = 'static/results/'

@app.route('/')
def index():
    return render_template('index.html')

@app.route('/', methods=['POST'])
def upload_image():
    file = request.files['file']
    if file :
        filename = file.filename
        filepath = os.path.join(app.config['UPLOAD_FOLDER'], filename)
        file.save(filepath)
        result_path = image_detect(filepath)
        if result_path:
            result_filename = os.path.basename(result_path)
            return jsonify({'filename': result_filename})
    return jsonify({'error': 'Image detection failed'})

@app.route('/video')
def video():
    #return Response(generate_frames(path=mavideo), mimetype='multipart/x-mixed-replace; boundary=frame')
    pass
    
@app.route('/webcam')
def webcam():
    return Response(generate_frames(path=0), mimetype='multipart/x-mixed-replace; boundary=frame')

@app.route('/gallery')
def gallery():
    images = os.listdir('static/results')
    images = [image for image in images if image.endswith(('jpg', 'jpeg', 'png', 'gif', 'JPG'))]
    return render_template('gallery.html', images=images)

if __name__ == '__main__':
    app.run(debug=False)

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [17/May/2024 12:52:02] "GET / HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2024 12:52:02] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /gallery HTTP/1.1" 200 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/style.css HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155150.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155202.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/IMG_4408.jpg HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155205.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155206.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155211.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/results/P4155212.JPG HTTP/1.1" 304 -
127.0.0.1 - - [17/May/2024 12:52:10] "GET /static/resul

: 